In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

In [109]:
def standardize(data: pd.Series):
    mean = data.mean()
    std = data.std()

    return data.apply(lambda x: (x - mean) / std)


In [ ]:
data = pd.read_csv("Dummy-Data-HSS.csv")
data.drop(columns=["Influencer"], inplace=True)
data.fillna(0, inplace=True)

for label in data.columns[:-1]:
    data[label] = standardize(data[label])

data.head(10)

In [ ]:
data_size = len(data)
test_size = int(data_size * 0.8)
val_size = data_size - test_size

test_data = data[:test_size]
val_data = data[test_size:]